# 03. Comparisons and Filtering

Master the **NEW ergonomic VectorOps API** for comparisons, filtering, and boolean operations. This notebook showcases the most math-first, clean comparison syntax available in RustLab.

## 🔥 What's New
- **Ergonomic scalar comparisons**: `v.gt(3.0)` instead of `v.gt_scalar(3.0)`
- **Clear vector comparisons**: `v1.gt_vec(&v2)` for element-wise
- **Single trait**: `VectorOps` replaces multiple comparison traits
- **Math-first syntax**: Closest to mathematical notation possible in Rust

## Setup

**Important**: The setup cell below follows Rust notebook best practices:
- Dependencies and imports are declared at the **top level** (outside braces) so they persist across all cells
- Test code is wrapped in braces `{}` to avoid persistence issues with complex types
- This pattern ensures compatibility with both rust-analyzer and evcxr

In [2]:
// Setup Cell - dependencies and imports persist across all cells
:dep rustlab-math = { path = ".." }

// Top-level imports - these persist across all cells!
use rustlab_math::*;

// Test setup in braces (variables don't persist, but confirms setup works)
{
    let test_vec = vec64![1.0, 2.0, 3.0];
    println!("✅ Setup complete! Test vector length: {}", test_vec.len());
}

✅ Setup complete! Test vector length: 3


()

## 1. Ergonomic Scalar Comparisons (NEW API)

The most common comparison operations - clean and intuitive:

In [3]:
// Sample data for analysis
let data = vec64![1.0, 2.0, 3.0, 4.0, 5.0];
let threshold = 3.0;

println!("Data: {:?}", data.to_slice());
println!("Threshold: {}", threshold);

Data: [1.0, 2.0, 3.0, 4.0, 5.0]
Threshold: 3


In [4]:
// NEW ergonomic scalar comparisons
let above_threshold = data.gt(threshold);      // v > 3.0
let below_four = data.lt(4.0);                 // v < 4.0  
let at_least_two = data.ge(2.0);               // v >= 2.0
let at_most_four = data.le(4.0);               // v <= 4.0
let equals_two = data.eq(2.0);                 // v == 2.0
let not_three = data.ne(3.0);                  // v != 3.0

println!("data.gt({}) (> threshold): {:?}", threshold, above_threshold.as_slice());
println!("data.lt(4.0) (< 4): {:?}", below_four.as_slice());
println!("data.ge(2.0) (≥ 2): {:?}", at_least_two.as_slice());
println!("data.eq(2.0) (== 2): {:?}", equals_two.as_slice());

data.gt(3) (> threshold): [false, false, false, true, true]
data.lt(4.0) (< 4): [true, true, true, false, false]
data.ge(2.0) (≥ 2): [false, true, true, true, true]
data.eq(2.0) (== 2): [false, true, false, false, false]


## 2. Vector-to-Vector Comparisons (NEW API)

Element-wise comparisons between vectors:

In [5]:
// Two vectors for comparison
let v1 = vec64![1.0, 2.0, 3.0, 4.0, 5.0];
let v2 = vec64![1.0, 3.0, 2.0, 4.0, 6.0];

println!("v1: {:?}", v1.to_slice());
println!("v2: {:?}", v2.to_slice());

v1: [1.0, 2.0, 3.0, 4.0, 5.0]
v2: [1.0, 3.0, 2.0, 4.0, 6.0]


In [6]:
// NEW vector comparison methods
let eq_mask = v1.eq_vec(&v2);                  // v1 == v2 element-wise
let gt_mask = v1.gt_vec(&v2);                  // v1 > v2 element-wise
let lt_mask = v1.lt_vec(&v2);                  // v1 < v2 element-wise
let ge_mask = v1.ge_vec(&v2);                  // v1 >= v2 element-wise

println!("v1.eq_vec(&v2) (equal): {:?}", eq_mask.as_slice());
println!("v1.gt_vec(&v2) (v1 > v2): {:?}", gt_mask.as_slice());
println!("v1.lt_vec(&v2) (v1 < v2): {:?}", lt_mask.as_slice());
println!("v1.ge_vec(&v2) (v1 ≥ v2): {:?}", ge_mask.as_slice());

v1.eq_vec(&v2) (equal): [true, false, false, true, false]
v1.gt_vec(&v2) (v1 > v2): [false, false, true, false, false]
v1.lt_vec(&v2) (v1 < v2): [false, true, false, false, true]
v1.ge_vec(&v2) (v1 ≥ v2): [true, false, true, true, false]


## 3. Floating-Point Safe Comparisons (RECOMMENDED)

Always use `.is_close()` for floating-point comparisons:

In [7]:
// Floating-point precision issues
let v3 = vec64![1.0, 2.000001, 3.0, 4.0, 5.0];
let v4 = vec64![1.0, 2.0, 3.00001, 4.0, 5.0];

// Exact equality (problematic for floating-point)
let exact = v3.eq_vec(&v4);
println!("Exact equality: {:?}", exact.as_slice());

// RECOMMENDED: Fuzzy equality with tolerance
let close = v3.is_close(&v4, 1e-5, 1e-8);     // rtol=1e-5, atol=1e-8
println!("Fuzzy equality: {:?}", close.as_slice());

println!("\n⚠️ Always use .is_close() for floating-point comparisons!");

Exact equality: [true, false, false, true, true]
Fuzzy equality: [true, true, true, true, true]

⚠️ Always use .is_close() for floating-point comparisons!


## 4. Boolean Logic Operations

Combine comparison results with boolean operators:

In [8]:
let data = vec64![1.0, 2.0, 3.0, 4.0, 5.0];

let above_three = data.gt(3.0);
let below_four = data.lt(4.0);

println!("above_three: {:?}", above_three.as_slice());
println!("below_four: {:?}", below_four.as_slice());

above_three: [false, false, false, true, true]
below_four: [true, true, true, false, false]


In [9]:
// Boolean logic operations
let both = above_three.and(&below_four).unwrap();      // AND: > 3 AND < 4
let either = above_three.or(&below_four).unwrap();     // OR: > 3 OR < 4
let not_above = above_three.clone().not();             // NOT: NOT (> 3)
let exclusive = above_three.xor(&below_four).unwrap(); // XOR: (> 3) XOR (< 4)

println!("AND (both conditions): {:?}", both.as_slice());
println!("OR (either condition): {:?}", either.as_slice());
println!("NOT (negation): {:?}", not_above.as_slice());
println!("XOR (exclusive): {:?}", exclusive.as_slice());

AND (both conditions): [false, false, false, false, false]
OR (either condition): [true, true, true, true, true]
NOT (negation): [true, true, true, false, false]
XOR (exclusive): [true, true, true, true, true]


## 5. Math-First Compound Conditions

Chain conditions with built-in operators for natural syntax:

In [10]:
let data = vec64![1.0, 2.0, 3.0, 4.0, 5.0];

// Range filtering with natural operators
let in_range = data.ge(2.0) & data.le(4.0);           // 2.0 ≤ x ≤ 4.0
let out_of_range = data.lt(2.0) | data.gt(4.0);       // x < 2.0 OR x > 4.0
let not_exactly_three = !data.eq(3.0);                // NOT (x == 3.0)

println!("Original data: {:?}", data.to_slice());
println!("In range [2, 4]: {:?}", in_range.unwrap().as_slice());
println!("Out of range: {:?}", out_of_range.unwrap().as_slice());
println!("Not exactly 3: {:?}", not_exactly_three.as_slice());

Original data: [1.0, 2.0, 3.0, 4.0, 5.0]
In range [2, 4]: [false, true, true, true, false]
Out of range: [true, false, false, false, true]
Not exactly 3: [true, true, false, true, true]


## 6. Statistical Analysis on Boolean Vectors

Extract insights from comparison results:

In [11]:
let data = vec64![1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0];
let outliers = data.gt(5.0);

// Statistical analysis
let has_any_outliers = outliers.any();                // Any element is true?
let all_are_outliers = outliers.all();                // All elements are true?
let outlier_count = outliers.count_true();            // Number of outliers
let normal_count = outliers.count_false();            // Number of normal values
let outlier_percentage = (outlier_count as f64) / (data.len() as f64) * 100.0;

println!("Data: {:?}", data.to_slice());
println!("Outliers (> 5): {:?}", outliers.as_slice());
println!("Any outliers? {}", has_any_outliers);
println!("All outliers? {}", all_are_outliers);
println!("Outlier count: {} ({:.1}%)", outlier_count, outlier_percentage);
println!("Normal count: {}", normal_count);

Data: [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0]
Outliers (> 5): [false, false, false, false, false, true, true, true]
Any outliers? true
All outliers? false
Outlier count: 3 (37.5%)


## 7. Index Extraction - Find Positions

Get the indices where conditions are true or false:

In [12]:
let scores = vec64![85.0, 92.0, 78.0, 96.0, 88.0, 74.0, 91.0];
let passing_grade = 80.0;

let passing = scores.ge(passing_grade);

// Extract indices
let passing_indices = passing.where_true();           // Indices of passing scores
let failing_indices = passing.where_false();          // Indices of failing scores

println!("Scores: {:?}", scores.to_slice());
println!("Passing (≥ {}): {:?}", passing_grade, passing.as_slice());
println!("Passing student indices: {:?}", passing_indices);
println!("Failing student indices: {:?}", failing_indices);

Normal count: 5
Scores: [85.0, 92.0, 78.0, 96.0, 88.0, 74.0, 91.0]
Passing (≥ 80): [true, true, false, true, true, false, true]
Passing student indices: [0, 1, 3, 4, 6]
Failing student indices: [2, 5]


## 8. Data Filtering - Extract Values (CRITICAL FEATURE)

The most important operation: extract actual values where conditions are true:

In [13]:
let temperatures = vec64![18.5, 22.3, 25.8, 19.1, 24.6, 28.2, 21.7, 26.4];
let comfortable_min = 20.0;
let comfortable_max = 25.0;

println!("All temperatures: {:?}", temperatures.to_slice());


// Create masks
let too_cold = temperatures.lt(comfortable_min);
let too_hot = temperatures.gt(comfortable_max);
let comfortable = temperatures.ge(comfortable_min) & temperatures.le(comfortable_max);

// Extract actual values
let cold_temps = temperatures.where_mask(&too_cold).unwrap();
let hot_temps = temperatures.where_mask(&too_hot).unwrap();
let good_temps = temperatures.where_mask(&comfortable.unwrap()).unwrap();

println!("Too cold (< {}): {:?}", comfortable_min, cold_temps);
println!("Too hot (> {}): {:?}", comfortable_max, hot_temps);
println!("Comfortable: {:?}", good_temps);

All temperatures: [18.5, 22.3, 25.8, 19.1, 24.6, 28.2, 21.7, 26.4]
Too cold (< 20): [18.5, 19.1]
Too hot (> 25): [25.8, 28.2, 26.4]
Comfortable: [22.3, 24.6, 21.7]


## 9. Real-World Example: Stock Analysis

Practical application of the VectorOps API for financial data analysis:

In [14]:
// Stock price data
let stock_prices = vec64![45.2, 52.1, 48.7, 55.3, 41.8, 49.2, 53.7, 38.5, 56.8, 50.1];
let target_price = 50.0;
let volatility_threshold = 5.0;

println!("Stock prices: {:?}", stock_prices.to_slice());
println!("Target price: {}", target_price);
println!("Volatility threshold: ±{}", volatility_threshold);

Stock prices: [45.2, 52.1, 48.7, 55.3, 41.8, 49.2, 53.7, 38.5, 56.8, 50.1]
Target price: 50
Volatility threshold: ±5


In [15]:
// Analysis using VectorOps
let above_target = stock_prices.gt(target_price);
let high_performers = stock_prices.gt(target_price + volatility_threshold);
let bargains = stock_prices.lt(target_price - volatility_threshold);
let stable_range = stock_prices.ge(target_price - 2.0) & stock_prices.le(target_price + 2.0);

// Extract actual stock prices
let strong_stocks = stock_prices.where_mask(&high_performers).unwrap();
let bargain_prices = stock_prices.where_mask(&bargains).unwrap();
let stable_prices = stock_prices.where_mask(&stable_range.unwrap()).unwrap();

println!("\n📊 Stock Analysis Results:");
println!("Above target ({}): {} stocks", target_price, above_target.count_true());
println!("High performers (> {}): {:?}", target_price + volatility_threshold, strong_stocks);
println!("Bargain opportunities (< {}): {:?}", target_price - volatility_threshold, bargain_prices);
println!("Stable prices (±2.0 range): {:?}", stable_prices);


📊 Stock Analysis Results:
Above target (50): 5 stocks
High performers (> 55): [55.3, 56.8]
Bargain opportunities (< 45): [41.8, 38.5]
Stable prices (±2.0 range): [48.7, 49.2, 50.1]


## 10. Advanced Filtering Patterns

Combine multiple conditions for sophisticated data analysis:

In [16]:
// Sensor data analysis
let sensor_data = vec64![22.5, 23.8, 19.2, 25.1, 18.7, 24.3, 26.8, 21.9, 17.5, 23.1];
let normal_min = 20.0;
let normal_max = 25.0;
let critical_low = 18.0;
let critical_high = 26.0;

println!("Sensor readings: {:?}", sensor_data.to_slice());

Sensor readings: [22.5, 23.8, 19.2, 25.1, 18.7, 24.3, 26.8, 21.9, 17.5, 23.1]


In [17]:
// Multi-level classification
let normal = sensor_data.ge(normal_min) & sensor_data.le(normal_max);
let warning_low = sensor_data.ge(critical_low) & sensor_data.lt(normal_min);
let warning_high = sensor_data.gt(normal_max) & sensor_data.le(critical_high);
let critical = sensor_data.lt(critical_low) | sensor_data.gt(critical_high);

// Extract values for each category
let normal_readings = sensor_data.where_mask(&normal.unwrap()).unwrap();
let warning_low_readings = sensor_data.where_mask(&warning_low.unwrap()).unwrap();
let warning_high_readings = sensor_data.where_mask(&warning_high.unwrap()).unwrap();
let critical_readings = sensor_data.where_mask(&critical.unwrap()).unwrap();

println!("\n🔍 Sensor Analysis:");
println!("Normal ({}-{}): {:?}", normal_min, normal_max, normal_readings);
println!("Warning Low: {:?}", warning_low_readings);
println!("Warning High: {:?}", warning_high_readings);
println!("Critical: {:?}", critical_readings);



🔍 Sensor Analysis:
Normal (20-25): [22.5, 23.8, 24.3, 21.9, 23.1]
Warning Low: [19.2, 18.7]
Warning High: [25.1]
Critical: [26.8, 17.5]


## 12. Summary - VectorOps API

### 🔥 Key Features
1. **Ergonomic scalar comparisons**: `v.gt(3.0)` - shorter, cleaner
2. **Clear vector comparisons**: `v1.gt_vec(&v2)` - unambiguous 
3. **Fuzzy equality**: `v1.is_close(&v2, rtol, atol)` - floating-point safe
4. **Data filtering**: `v.where_mask(&mask)` - extract matching values
5. **Boolean logic**: `&`, `|`, `!` operators work naturally
6. **Statistical analysis**: `.any()`, `.all()`, `.count_true()`

### 🎯 Best Practices
1. Use `v.gt(scalar)` for thresholding and outlier detection
2. Use `v1.gt_vec(&v2)` for element-wise vector comparisons
3. Always use `.is_close()` instead of `.eq_vec()` for floating-point data
4. Chain conditions with `&`, `|`, `!` for natural mathematical expressions
5. Extract filtered data with `.where_mask(&condition)`

### ⚡ Advantages
- **More ergonomic** than old API
- **Math-first syntax** - closest to mathematical notation
- **Single trait** - simpler imports
- **Consistent** with NumPy and scientific libraries
- **Comprehensive** fuzzy equality support

Next notebook: Linear Algebra operations